# Try 16: setup `paprika` restraints

Presuming we have a host-guest system already setup, let's see if we can rationally instruct `paprika` to setup all the restraints for a real system.

In [1]:
%load_ext autoreload
%autoreload 2

import os as os
import numpy as np

import parmed as pmd
import pytraj as pt

In [2]:
import paprika
print(paprika.__version__)

from paprika.restraints import static_DAT_restraint
from paprika.restraints import DAT_restraint
from paprika.restraints import amber_restraint_line
from paprika.restraints import create_window_list

from paprika.utils import make_window_dirs

2018-03-29 06:35:09 PM OpenMM support: Yes


2018-03-29_11:00:47_-0700-9c2dad8-0.0.3


## Preamble

We are going to setup host translational restraints, guest translational restraints, host "jacks," and guest wall restraints with `paprika`. First, let's try working in a directory where we have a known `disang.rest` file we can use as a reference: `/data/davids4/projects/smirnoff-host-guest/a-bam-p/a00/original` or `/data/davids4/projects/smirnoff-host-guest/a-bam-p/u00/original` for the final values.

Since we're starting with files Niel prepared, the dummy atoms are first, followed by the host, and then the guest atoms.

In [3]:
dummy_anchors = [':1', ':2', ':3']
host_anchors  = [':4@O3', ':6@C1', ':8@C6']
guest_anchors = [':10@C4', ':10@N1']

In [4]:
hg = pmd.load_file('systems/a-bam-p/a00/original/full.hmr.topo',
                   'systems/a-bam-p/a00/original/full.crds',
                    structure=True)

The attach fractions can be found in the `Setup.pl` file in the original location on `kirkwood`: `/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/a-bam-p/a00/` under `@AttachFC`.

The attach force constant is 5.0 kcal/mol-A^{2} for distances and 100.0 kcal/mol-rad^{2} for angels and torsions based on `disang.rest` in `u00`.

The pull distances are specified by `@Translate` in `Setup.pl` and are offset by 6 Angstroms.

In [5]:
attach_string = '0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 18.10 24.40 37.00 49.60 74.80 100.00'
attach_fractions = [float(i) / 100 for i in attach_string.split()]

pull_string = '0.00 0.40 0.80 1.20 1.60 2.00 2.40 2.80 3.20 3.60 4.00 4.40 4.80 5.20 5.60 6.00 6.40 6.80 7.20 7.60 8.00 8.40 8.80 9.20 9.60 10.00 10.40 10.80 11.20 11.60 12.00 12.40 12.80 13.20 13.60 14.00 14.40 14.80 15.20 15.60 16.00 16.40 16.80 17.20 17.60 18.00'
pull_distances = [float(i) + 6.00 for i in pull_string.split()]

windows = [len(attach_fractions), len(pull_distances), 0]
print(f'There are {windows} windows in this attach-pull calculation.')

There are [15, 46, 0] windows in this attach-pull calculation.


## Define the host translational restraints
These restraints help pin down the host during initial attachment of the DAT restraints. They are static, meaning unlike other restraints, they are not turned on slowly during the attachment phase.

In [6]:
static_restraint_atoms = [[dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[2], dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1], host_anchors[2]],
                        ]

static_restraint_distance_fc = 5.0
static_restraint_angle_fc = 100.0

## Define the guest translational restraints

In [7]:
guest_restraint_atoms = [[dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[1], dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[0], guest_anchors[0], guest_anchors[1]],
                        ]

guest_restraint_targets = [6.0,
                           180.0,
                           180.0
                          ]
guest_restraint_distance_fc = 5.0
guest_restraint_angle_fc = 100.0

## Define the host conformational restraints

The "jacks" restraints can be written O5_n - C1_n - O1_n - C4_n+1 and C1_n - O1_n - C4_n+1 - C5_n+1.

In [8]:
host_conformational_template = [['O5', 'C1', 'O1', 'C4'],
                                ['C1', 'O1', 'C4', 'C5']
                               ]

host_residues = len(hg[':MGO'].residues)
first_host_residue = hg[':MGO'].residues[0].number + 1
conformational_restraint_atoms = []
conformational_restraint_targets = []
conformational_restraint_fc = 6.0

for n in range(first_host_residue, host_residues + first_host_residue):
    if n + 1 < host_residues + first_host_residue:
        next_residue = n + 1
    else:
        next_residue = first_host_residue
    conformational_restraint_atoms.append(
        [f':{n}@{host_conformational_template[0][0]}',
         f':{n}@{host_conformational_template[0][1]}',
         f':{n}@{host_conformational_template[0][2]}',
         f':{next_residue}@{host_conformational_template[0][3]}'
        ])
    conformational_restraint_targets.append(104.30)
    conformational_restraint_atoms.append(
        [f':{n}@{host_conformational_template[1][0]}',
         f':{n}@{host_conformational_template[1][1]}',
         f':{next_residue}@{host_conformational_template[1][2]}',
         f':{next_residue}@{host_conformational_template[1][3]}'
        ])
    conformational_restraint_targets.append(-108.8)

## Define the guest "wall" restraints

The guest wall restraints are only on during the attachment phase, and make sure that the guest does not unbind and waste time diffusing around the simulation box. These distance restraints are flat from zero through the restraint target value, near the edge of the binding cavity, and then increase harmonically beyond that. The guest "wall" angle restraint is harmonic and prevents the guest from flipping around during the bound phase.

In [9]:
guest_wall_template = [
    ['HO2', guest_anchors[0]],
    ['HO6', guest_anchors[0]]
]

guest_wall_restraint_atoms = []
guest_wall_restraint_targets = []
guest_wall_restraint_angle_fc = 500.0
guest_wall_restraint_distance_fc = 50.0

for n in range(first_host_residue, host_residues + first_host_residue):
    guest_wall_restraint_atoms.append(
        [f':{n}@{guest_wall_template[0][0]}',
         f'{guest_wall_template[0][1]}',
        ])
    guest_wall_restraint_targets.append(11.3)
    guest_wall_restraint_atoms.append(
        [f':{n}@{guest_wall_template[1][0]}',
         f'{guest_wall_template[1][1]}',
        ])
    guest_wall_restraint_targets.append(13.3)

    
guest_wall_restraint_atoms.append([dummy_anchors[1],
                                 guest_anchors[0],
                                 guest_anchors[1]])
guest_wall_restraint_targets.append(80.0)

## Initialize all the restraints

In [10]:
for index, atoms in enumerate(static_restraint_atoms):    
    static_DAT_restraint(restraint_mask_list=atoms,
                        num_window_list=windows, 
                        ref_structure=hg,
                        force_constant=static_restraint_angle_fc if len(atoms) > 2 else static_restraint_distance_fc,
                        amber_index=True)

2018-03-29 06:35:10 PM Calculating attach targets and force constants...
2018-03-29 06:35:10 PM Attach, Method #1
2018-03-29 06:35:10 PM Calculating pull targets and force constants...
2018-03-29 06:35:10 PM Pull, Method #1
2018-03-29 06:35:10 PM Calculating release targets and force constants...
2018-03-29 06:35:10 PM No restraint info set for the release phase! Skipping...
2018-03-29 06:35:10 PM Number of attach windows = 15
2018-03-29 06:35:10 PM Number of pull windows = 46
2018-03-29 06:35:10 PM This restraint will be skipped in the release phase
2018-03-29 06:35:10 PM Assigning atom indices...
2018-03-29 06:35:10 PM There are 1 atoms in the mask :1  ...
2018-03-29 06:35:10 PM There are 1 atoms in the mask :4@O3  ...
2018-03-29 06:35:11 PM Calculating attach targets and force constants...
2018-03-29 06:35:11 PM Attach, Method #1
2018-03-29 06:35:11 PM Calculating pull targets and force constants...
2018-03-29 06:35:11 PM Pull, Method #1
2018-03-29 06:35:11 PM Calculating release ta

In [11]:
for index, atoms in enumerate(guest_restraint_atoms): 
    if len(atoms) > 2:
        angle = True
    else:
        angle = False
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    if angle:
        this.mask3 = atoms[2]
        this.attach['fc_final'] = guest_restraint_angle_fc
    else:
        this.attach['fc_final'] = guest_restraint_distance_fc
    this.attach['target'] = guest_restraint_targets[index]
    this.attach['fraction_list'] = attach_fractions
    this.pull['target_final'] = guest_restraint_targets[index]
    this.pull['num_windows'] = windows[1]
    this.initialize()

2018-03-29 06:35:12 PM Calculating attach targets and force constants...
2018-03-29 06:35:12 PM Attach, Method #3
2018-03-29 06:35:12 PM Calculating pull targets and force constants...
2018-03-29 06:35:12 PM Pull, Method #1
2018-03-29 06:35:12 PM Calculating release targets and force constants...
2018-03-29 06:35:12 PM No restraint info set for the release phase! Skipping...
2018-03-29 06:35:12 PM Number of attach windows = 15
2018-03-29 06:35:12 PM Number of pull windows = 46
2018-03-29 06:35:12 PM This restraint will be skipped in the release phase
2018-03-29 06:35:12 PM Assigning atom indices...
2018-03-29 06:35:12 PM There are 1 atoms in the mask :1  ...
2018-03-29 06:35:12 PM There are 1 atoms in the mask :10@C4  ...
2018-03-29 06:35:12 PM Calculating attach targets and force constants...
2018-03-29 06:35:12 PM Attach, Method #3
2018-03-29 06:35:12 PM Calculating pull targets and force constants...
2018-03-29 06:35:12 PM Pull, Method #1
2018-03-29 06:35:12 PM Calculating release t

In [12]:
for index, atoms in enumerate(conformational_restraint_atoms): 
    
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    this.mask3 = atoms[2]
    this.mask4 = atoms[3]
    
    this.attach['fraction_list'] = attach_fractions
    this.attach['target'] = conformational_restraint_targets[index]
    this.attach['fc_final'] = conformational_restraint_fc
    this.pull['target_final'] = conformational_restraint_targets[index]
    this.pull['num_windows'] = windows[1]
    
    this.initialize()

2018-03-29 06:35:13 PM Calculating attach targets and force constants...
2018-03-29 06:35:13 PM Attach, Method #3
2018-03-29 06:35:13 PM Calculating pull targets and force constants...
2018-03-29 06:35:13 PM Pull, Method #1
2018-03-29 06:35:13 PM Calculating release targets and force constants...
2018-03-29 06:35:13 PM No restraint info set for the release phase! Skipping...
2018-03-29 06:35:13 PM Number of attach windows = 15
2018-03-29 06:35:13 PM Number of pull windows = 46
2018-03-29 06:35:13 PM This restraint will be skipped in the release phase
2018-03-29 06:35:13 PM Assigning atom indices...
2018-03-29 06:35:13 PM There are 1 atoms in the mask :4@O5  ...
2018-03-29 06:35:13 PM There are 1 atoms in the mask :4@C1  ...
2018-03-29 06:35:13 PM There are 1 atoms in the mask :4@O1  ...
2018-03-29 06:35:13 PM There are 1 atoms in the mask :5@C4  ...
2018-03-29 06:35:13 PM Calculating attach targets and force constants...
2018-03-29 06:35:13 PM Attach, Method #3
2018-03-29 06:35:13 PM C

2018-03-29 06:35:14 PM This restraint will be skipped in the release phase
2018-03-29 06:35:14 PM Assigning atom indices...
2018-03-29 06:35:14 PM There are 1 atoms in the mask :8@C1  ...
2018-03-29 06:35:14 PM There are 1 atoms in the mask :8@O1  ...
2018-03-29 06:35:14 PM There are 1 atoms in the mask :9@C4  ...
2018-03-29 06:35:14 PM There are 1 atoms in the mask :9@C5  ...
2018-03-29 06:35:14 PM Calculating attach targets and force constants...
2018-03-29 06:35:14 PM Attach, Method #3
2018-03-29 06:35:14 PM Calculating pull targets and force constants...
2018-03-29 06:35:14 PM Pull, Method #1
2018-03-29 06:35:14 PM Calculating release targets and force constants...
2018-03-29 06:35:14 PM No restraint info set for the release phase! Skipping...
2018-03-29 06:35:14 PM Number of attach windows = 15
2018-03-29 06:35:14 PM Number of pull windows = 46
2018-03-29 06:35:14 PM This restraint will be skipped in the release phase
2018-03-29 06:35:14 PM Assigning atom indices...
2018-03-29 06:

In [13]:
for index, atoms in enumerate(guest_wall_restraint_atoms): 
    if len(atoms) > 2:
        angle = True
    else:
        angle = False
    
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    if angle:
        this.mask3 = atoms[2]
        this.attach['fc_final'] = guest_wall_restraint_angle_fc
        this.custom_restraint_values['rk3'] = 0.0
    else:
        this.attach['fc_final'] = guest_wall_restraint_distance_fc
        this.custom_restraint_values['rk3'] = 50.0
        this.custom_restraint_values['rk4'] = 50.0
        this.custom_restraint_values['r1'] = 0.0
        this.custom_restraint_values['r2'] = 0.0

    this.attach['target'] = guest_wall_restraint_targets[index]
    this.attach['fraction_list'] = attach_fractions
    this.pull['target_final'] = guest_wall_restraint_targets[index]
    this.pull['num_windows'] = windows[1]
    this.initialize()

2018-03-29 06:35:15 PM Calculating attach targets and force constants...
2018-03-29 06:35:15 PM Attach, Method #3
2018-03-29 06:35:15 PM Calculating pull targets and force constants...
2018-03-29 06:35:15 PM Pull, Method #1
2018-03-29 06:35:15 PM Calculating release targets and force constants...
2018-03-29 06:35:15 PM No restraint info set for the release phase! Skipping...
2018-03-29 06:35:15 PM Number of attach windows = 15
2018-03-29 06:35:15 PM Number of pull windows = 46
2018-03-29 06:35:15 PM This restraint will be skipped in the release phase
2018-03-29 06:35:15 PM Assigning atom indices...
2018-03-29 06:35:15 PM There are 1 atoms in the mask :4@HO2  ...
2018-03-29 06:35:15 PM There are 1 atoms in the mask :10@C4  ...
2018-03-29 06:35:15 PM Calculating attach targets and force constants...
2018-03-29 06:35:15 PM Attach, Method #3
2018-03-29 06:35:15 PM Calculating pull targets and force constants...
2018-03-29 06:35:15 PM Pull, Method #1
2018-03-29 06:35:15 PM Calculating relea

2018-03-29 06:35:16 PM Calculating pull targets and force constants...
2018-03-29 06:35:16 PM Pull, Method #1
2018-03-29 06:35:16 PM Calculating release targets and force constants...
2018-03-29 06:35:16 PM No restraint info set for the release phase! Skipping...
2018-03-29 06:35:16 PM Number of attach windows = 15
2018-03-29 06:35:16 PM Number of pull windows = 46
2018-03-29 06:35:16 PM This restraint will be skipped in the release phase
2018-03-29 06:35:16 PM Assigning atom indices...
2018-03-29 06:35:16 PM There are 1 atoms in the mask :9@HO6  ...
2018-03-29 06:35:16 PM There are 1 atoms in the mask :10@C4  ...
2018-03-29 06:35:16 PM Calculating attach targets and force constants...
2018-03-29 06:35:16 PM Attach, Method #3
2018-03-29 06:35:16 PM Calculating pull targets and force constants...
2018-03-29 06:35:16 PM Pull, Method #1
2018-03-29 06:35:16 PM Calculating release targets and force constants...
2018-03-29 06:35:16 PM No restraint info set for the release phase! Skipping...


## Make the window list

In [14]:
window_list = create_window_list(DAT_restraint.instances)

2018-03-29 06:35:16 PM All restraints are "continuous_apr" style.
2018-03-29 06:35:16 PM Restraints appear to be consistent


In [15]:
make_window_dirs(window_list)

In [16]:
for window in window_list:
    with open('windows/' + window + '/disang.rest', 'w') as file:
        for restraint in DAT_restraint.instances:
            if window[0] == 'a':
                phase = 'attach'
            if window[0] == 'p':
                phase = 'pull'
            if window[0] == 'r':
                phase = 'release'
            print(amber_restraint_line(restraint, phase, int(window[1:])))
            file.write(amber_restraint_line(restraint, phase, int(window[1:])))

2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overrid

2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overridin

&rst iat = 1,13,             r1 =    0.00000, r2 =    7.49023, r3 =    7.49023, r4 =  999.00000, rk2 =    5.00000, rk3 =    5.00000,  &end

&rst iat = 2,1,13,           r1 =    0.00000, r2 =  131.36554, r3 =  131.36554, r4 =  180.00000, rk2 =  100.00000, rk3 =  100.00000,  &end

&rst iat = 3,2,1,13,         r1 =  -64.88459, r2 =  115.11541, r3 =  115.11541, r4 =  295.11541, rk2 =  100.00000, rk3 =  100.00000,  &end

&rst iat = 1,13,46,          r1 =    0.00000, r2 =   55.11100, r3 =   55.11100, r4 =  180.00000, rk2 =  100.00000, rk3 =  100.00000,  &end

&rst iat = 2,1,13,46,        r1 =  -15.40895, r2 =  164.59105, r3 =  164.59105, r4 =  344.59105, rk2 =  100.00000, rk3 =  100.00000,  &end

&rst iat = 1,13,46,104,      r1 =  -70.84632, r2 =  109.15368, r3 =  109.15368, r4 =  289.15368, rk2 =  100.00000, rk3 =  100.00000,  &end

&rst iat = 1,143,            r1 =    0.00000, r2 =    6.00000, r3 =    6.00000, r4 =  999.00000, rk2 =    0.00000, rk3 =    0.00000,  &end

&rst iat = 2,1,143, 

2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overridin

2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding rk3 = 50.0
2018-03-29 06:35:16 PM Overriding rk4 = 50.0
2018-03-29 06:35:16 PM Overriding r1 = 0.0
2018-03-29 06:35:16 PM Overriding r2 = 0.0
2018-03-29 06:35:16 PM Overriding 

...

2018-03-29 06:35:19 PM Overriding rk4 = 50.0
2018-03-29 06:35:19 PM Overriding r1 = 0.0
2018-03-29 06:35:19 PM Overriding r2 = 0.0
2018-03-29 06:35:19 PM Overriding rk3 = 0.0
2018-03-29 06:35:19 PM Overriding rk3 = 0.0
2018-03-29 06:35:19 PM Overriding rk3 = 50.0
2018-03-29 06:35:19 PM Overriding rk4 = 50.0
2018-03-29 06:35:19 PM Overriding r1 = 0.0
2018-03-29 06:35:19 PM Overriding r2 = 0.0
2018-03-29 06:35:19 PM Overriding rk3 = 50.0
2018-03-29 06:35:19 PM Overriding rk4 = 50.0
2018-03-29 06:35:19 PM Overriding r1 = 0.0
2018-03-29 06:35:19 PM Overriding r2 = 0.0
2018-03-29 06:35:19 PM Overriding rk3 = 50.0
2018-03-29 06:35:19 PM Overriding rk4 = 50.0
2018-03-29 06:35:19 PM Overriding r1 = 0.0
2018-03-29 06:35:19 PM Overriding r2 = 0.0
2018-03-29 06:35:19 PM Overriding rk3 = 50.0
2018-03-29 06:35:19 PM Overriding rk4 = 50.0
2018-03-29 06:35:19 PM Overriding r1 = 0.0
2018-03-29 06:35:19 PM Overriding r2 = 0.0
2018-03-29 06:35:19 PM Overriding rk3 = 50.0
2018-03-29 06:35:19 PM Overridin

2018-03-29 06:35:20 PM Overriding rk3 = 50.0
2018-03-29 06:35:20 PM Overriding rk4 = 50.0
2018-03-29 06:35:20 PM Overriding r1 = 0.0
2018-03-29 06:35:20 PM Overriding r2 = 0.0
2018-03-29 06:35:20 PM Overriding rk3 = 50.0
2018-03-29 06:35:20 PM Overriding rk4 = 50.0
2018-03-29 06:35:20 PM Overriding r1 = 0.0
2018-03-29 06:35:20 PM Overriding r2 = 0.0
2018-03-29 06:35:20 PM Overriding rk3 = 50.0
2018-03-29 06:35:20 PM Overriding rk4 = 50.0
2018-03-29 06:35:20 PM Overriding r1 = 0.0
2018-03-29 06:35:20 PM Overriding r2 = 0.0
2018-03-29 06:35:20 PM Overriding rk3 = 0.0
2018-03-29 06:35:20 PM Overriding rk3 = 0.0
2018-03-29 06:35:20 PM Overriding rk3 = 50.0
2018-03-29 06:35:20 PM Overriding rk4 = 50.0
2018-03-29 06:35:20 PM Overriding r1 = 0.0
2018-03-29 06:35:20 PM Overriding r2 = 0.0
2018-03-29 06:35:20 PM Overriding rk3 = 50.0
2018-03-29 06:35:20 PM Overriding rk4 = 50.0
2018-03-29 06:35:20 PM Overriding r1 = 0.0
2018-03-29 06:35:20 PM Overriding r2 = 0.0
2018-03-29 06:35:20 PM Overridin

&rst iat = 2,143,130,        r1 =    0.00000, r2 =   80.00000, r3 =   80.00000, r4 =  180.00000, rk2 =  500.00000, rk3 =    0.00000,  &end

&rst iat = 1,13,             r1 =    0.00000, r2 =    7.49023, r3 =    7.49023, r4 =  999.00000, rk2 =    5.00000, rk3 =    5.00000,  &end

&rst iat = 2,1,13,           r1 =    0.00000, r2 =  131.36554, r3 =  131.36554, r4 =  180.00000, rk2 =  100.00000, rk3 =  100.00000,  &end

&rst iat = 3,2,1,13,         r1 =  -64.88459, r2 =  115.11541, r3 =  115.11541, r4 =  295.11541, rk2 =  100.00000, rk3 =  100.00000,  &end

&rst iat = 1,13,46,          r1 =    0.00000, r2 =   55.11100, r3 =   55.11100, r4 =  180.00000, rk2 =  100.00000, rk3 =  100.00000,  &end

&rst iat = 2,1,13,46,        r1 =  -15.40895, r2 =  164.59105, r3 =  164.59105, r4 =  344.59105, rk2 =  100.00000, rk3 =  100.00000,  &end

&rst iat = 1,13,46,104,      r1 =  -70.84632, r2 =  109.15368, r3 =  109.15368, r4 =  289.15368, rk2 =  100.00000, rk3 =  100.00000,  &end

&rst iat = 1,143,   

2018-03-29 06:35:20 PM Overriding rk4 = 50.0
2018-03-29 06:35:20 PM Overriding r1 = 0.0
2018-03-29 06:35:20 PM Overriding r2 = 0.0
2018-03-29 06:35:20 PM Overriding rk3 = 50.0
2018-03-29 06:35:20 PM Overriding rk4 = 50.0
2018-03-29 06:35:20 PM Overriding r1 = 0.0
2018-03-29 06:35:20 PM Overriding r2 = 0.0
2018-03-29 06:35:20 PM Overriding rk3 = 50.0
2018-03-29 06:35:20 PM Overriding rk4 = 50.0
2018-03-29 06:35:20 PM Overriding r1 = 0.0
2018-03-29 06:35:20 PM Overriding r2 = 0.0
2018-03-29 06:35:20 PM Overriding rk3 = 50.0
2018-03-29 06:35:20 PM Overriding rk4 = 50.0
2018-03-29 06:35:20 PM Overriding r1 = 0.0
2018-03-29 06:35:20 PM Overriding r2 = 0.0
2018-03-29 06:35:20 PM Overriding rk3 = 50.0
2018-03-29 06:35:20 PM Overriding rk4 = 50.0
2018-03-29 06:35:20 PM Overriding r1 = 0.0
2018-03-29 06:35:20 PM Overriding r2 = 0.0
2018-03-29 06:35:20 PM Overriding rk3 = 50.0
2018-03-29 06:35:20 PM Overriding rk4 = 50.0
2018-03-29 06:35:20 PM Overriding r1 = 0.0
2018-03-29 06:35:20 PM Overridin

&rst iat = 87,143,           r1 =    0.00000, r2 =    0.00000, r3 =   13.30000, r4 =  999.00000, rk2 =   50.00000, rk3 =   50.00000,  &end

&rst iat = 94,143,           r1 =    0.00000, r2 =    0.00000, r3 =   11.30000, r4 =  999.00000, rk2 =   50.00000, rk3 =   50.00000,  &end

&rst iat = 108,143,          r1 =    0.00000, r2 =    0.00000, r3 =   13.30000, r4 =  999.00000, rk2 =   50.00000, rk3 =   50.00000,  &end

&rst iat = 115,143,          r1 =    0.00000, r2 =    0.00000, r3 =   11.30000, r4 =  999.00000, rk2 =   50.00000, rk3 =   50.00000,  &end

&rst iat = 129,143,          r1 =    0.00000, r2 =    0.00000, r3 =   13.30000, r4 =  999.00000, rk2 =   50.00000, rk3 =   50.00000,  &end

&rst iat = 2,143,130,        r1 =    0.00000, r2 =   80.00000, r3 =   80.00000, r4 =  180.00000, rk2 =  500.00000, rk3 =    0.00000,  &end



In [17]:
def compare_disang(reference, target):
 
    with open(reference, 'r') as ref:
        next(ref)
        reference_lines = ref.readlines()
    with open(target, 'r') as tar:
        target_lines = tar.readlines()
               
    # for x, y in zip(sorted(reference_lines), sorted(target_lines)):
    for x, y in zip(reference_lines, target_lines):
        
        x = x.strip().split()
        y = y.strip().split()
        
        ref_atoms = x[2][:-1]
        ref_r1 = x[4][:-1]
        ref_r2 = x[6][:-1]
        ref_r3 = x[8][:-1]
        ref_r4 = x[10][:-1]
        ref_rk2 = x[12][:-1]
        ref_rk3 = x[14][:-1]
            
        tar_atoms = y[3][:-1]
        tar_r1 = y[6][:-1]
        tar_r2 = y[9][:-1]
        tar_r3 = y[12][:-1]
        tar_r4 = y[15][:-1]
        tar_rk2 = y[18][:-1]
        tar_rk3 = y[21][:-1]
        
        # print(ref_atoms + ' ?=? ' + tar_atoms)
        # print(ref_r1 + ' ?=? ' + tar_r1)

        print('{:20s}{:20s}'.format(ref_atoms, tar_atoms))
        
        try:
            assert(ref_atoms == tar_atoms)
            # np.testing.assert_almost_equal(float(ref_r1), float(tar_r1), decimal=2)
            #np.testing.assert_almost_equal(float(ref_r2), float(tar_r2), decimal=2)
            #np.testing.assert_almost_equal(float(ref_r3), float(tar_r3), decimal=2)
            #np.testing.assert_almost_equal(float(ref_r4), float(tar_r4), decimal=2)
            #np.testing.assert_almost_equal(float(ref_rk2), float(tar_rk2), decimal=2)
            #np.testing.assert_almost_equal(float(ref_rk3), float(tar_rk2), decimal=2)
        except:
            pass
            # print('Mismatch!')
#             print(x)
#             print(y)
        

In [18]:
a, b = compare_disang('/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/a00/original/disang.rest',
              '/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/windows/a000/disang.rest')

1,13                1,13                
2,1,13              2,1,13              
3,2,1,13            3,2,1,13            
1,13,46             1,13,46             
2,1,13,46           2,1,13,46           
1,13,46,104         1,13,46,104         
1,143               1,143               
2,1,143             2,1,143             
1,143,130           1,143,130           
19,4,6,36           19,4,6,36           
40,25,27,57         4,6,36,38           
61,46,48,78         40,25,27,57         
82,67,69,99         25,27,57,59         
103,88,90,120       61,46,48,78         
124,109,111,15      46,48,78,80         
4,6,36,38           82,67,69,99         
25,27,57,59         67,69,99,101        
46,48,78,80         103,88,90,120       
67,69,99,101        88,90,120,122       
88,90,120,122       124,109,111,15      
109,111,15,17       109,111,15,17       
9,14                10,143              
23,14               24,143              
30,14               31,143              
44,14           

TypeError: 'NoneType' object is not iterable

In [ ]:
type(a)

In [ ]:
type(b)

In [ ]:
a[0]